In [38]:
import requests
import time
import datasets
import pandas as pd

In [2]:
# deploy teacher model
# llama-3.3-70b-instruct

url = "http://nemo.test/v1/deployment/model-deployments"  # if you used Launchable setup, then the base url will be http://nemo.test. Otherwise, change the base url accordingly.
headers = {
    "accept": "application/json",
    "Content-Type": "application/json",
}
payload = {
    "name": "llama-3.3-70b-instruct",
    "namespace": "meta",
    "config": {
        "model": "meta/llama-3.3-70b-instruct",
        "nim_deployment": {
            "image_name": "nvcr.io/nim/meta/llama-3.2-1b-instruct",
            "image_tag": "1.8.5",
            "pvc_size": "25Gi",
            "gpu": 2,
            "additional_envs": {
                "NIM_GUIDED_DECODING_BACKEND": "outlines"
            }
        }
    }
}

response = requests.post(url, headers=headers, json=payload)
print(response.status_code)
print(response.json())


200
{'async_enabled': False, 'config': {'model': 'meta/llama-3.3-70b-instruct', 'nim_deployment': {'additional_envs': {'NIM_GUIDED_DECODING_BACKEND': 'outlines'}, 'disable_lora_support': False, 'gpu': 2, 'image_name': 'nvcr.io/nim/meta/llama-3.2-1b-instruct', 'image_tag': '1.8.5', 'pvc_size': '25Gi'}}, 'created_at': '2025-10-23T17:11:26.311024045Z', 'deployed': False, 'name': 'llama-3.3-70b-instruct', 'namespace': 'meta', 'status_details': {'description': 'Model deployment created', 'status': 'pending'}, 'url': ''}


In [21]:
# List models
response = requests.get("http://nim.test/v1/models")
if response.status_code == 200:
    models = [m['id'] for m in response.json()['data']]
    print(f"Models: {models}") 

Models: ['meta/llama-3.3-70b-instruct']


In [ ]:
# test the deployment with a simple prompt
url = "http://nim.test/v1/chat/completions"

payload = {
    "model": "meta/llama-3.3-70b-instruct",
    "messages": [
        {"role": "user", "content": "What is the capital of France?"}
    ],
    "max_tokens": 200,
    "temperature": 0.7
}

response = requests.post(url, json=payload)
print(response.json()['choices'][0]['message']['content'])

The capital of France is Paris.


In [27]:
data = datasets.load_dataset("/home/shadeform/data-flywheel-fsi/data/news_data_raw")

In [32]:
PROMPT = """
You are a helpful AI assistant that analyses financial news headlines and identifies what event type is described.
You will classify event types into one of the following categories (in square brackets)

- [Analyst Rating]: An entity such as a bank, asset manager, etc. gives a classification/rating/downgrade/upgrade/opinion to an asset.
                    If there is no specified analyst and company given, it's not Analyst Rating and should be classified as OTHER.
- [Price Targets]: A mention of a price target (PT) is given by an entity such as bank, asset manager, etc.
                   This takes priority over any other class, so if a price target is present use this class!
- [Earnings]: Reports of quarterly, monthly, etc. concrete values of revenue, ESP, etc. Percentage fluctations.
              Expected values are not Earnings and should be Guidance instead!
- [Labour Issues]: Mentions of layoffs, union action, strikes, rising cost of labour, bonuses for execs, etc. Important personal change, e.g. CEO, CFO, VPs, etc.
- [Mergers and Acquisitions]: Whenever merging or acquisition of entities, not just companies, is mentioned.
                              Things like partnerships do not belong to this class! Takes priority over other classes.
- [Dividends]: Mentions of dividend performance, dividend per share, decisions not to issue dividends, etc.
- [Regulatory]: Mentions corporate position focused on environmental affairs, government regulation, international treaties, geopolitics, debt repayment, licenses, patents, etc.
                Any executive decisions taken by the government are Regulatory. Takes priority over other classes.
- [Stock price movement]: Includes pricing of public offerings, daily, monthly and yearly movements, highs and lows, options trade and alerts, etc.
                          Only relevant when a specific entity/industry or set of entities/industries mentioned.
                          Quantities such as sales, are not stock price movements. Stock splits do not count as stock movements.
                          Takes priority over Earnings, Stock Price Movement, Credit Ratings
- [Credit Ratings]: Adjustments of company's borrowing capacity, changes in debt values, changes in ratings, etc.
- [Products-Services]: Mentions of a company's particular product, forward-looking product directions, disruption, government and private contracts, etc.
                       Any sort of delay regarding a product roll-out, etc.
- [Product Approval]: Mentions FDA approvals, environmental approvals, acceptance for review. Any time an entity approves the rollout of a corporations product.
- [Guidance]: Forward looking statements issued by the company's themselves regarding Revenue, EPS, potential sales going up/down, number of contracts, etc.
              Similar talk to earnings but about projections rather than realised.
              
If the headline doesn't match any of the classes, classify it as OTHER.
ATTENTION:
  - Only assign a category if the headline meets all the criteria listed for that category. Otherwise use OTHER
  - Encourage precise matching rather than assigning categories based on partial or superficial similarities
  - OTHER is the default category when in doubt
  - If there are no specific companies mentioned, use OTHER

A few examples:

1. Cornerstone OnDemand Higher as Barclays Upgraded to Overweight -> [Analyst Rating]
2. HC Wainwright & Co. Maintains Buy on Balchem, Lowers Price Target to $104 -> [Price Targets]
3. Sina Reports Q4 Adj. EPS $0.24 vs $0.18 Est., Sales $211.1M vs $207.6M Est.; Sees FY15 Sales $800M-$900M vs $884.6M Est. -> [Earnings]
4. Delta To Buy Out Employees, Offer Early Retirement (DAL) -> [Labour Issues]
5. Constant Contact Announces Deal to Be Purchased by Endurance Int'l at $32/Share -> [Mergers and Acquisitions]
6. Ameriprise Financial Announces 12% Qtr. Dividend Increase to $0.58/Share -> [Dividends]
7. Ultratech Achieves ISO 9001 and 14001 Certification for Singapore Operations and Recertification for U.S. Facility  -> [Regulatory]
8. Mid-Afternoon Market Update: Dow Up Over 200 Points; Lakeland Industries Shares Spike Higher -> [Stock price movement]
9. Moody's Affirms Ratings on Nokia; Outlook Revised from Negative to Developing -> [Credit Ratings]
10. NOVAVAX Awarded HHS-BARDA Contract Valued at up to $179 Million  -> [Products-Services]
11. Sanofi's Genzyme Announces Lemtrada Resubmission Accepted for Review by FDA  -> [Product Approval]
12. Dot Hill Systems Announces Re-alignment of Software Development Initiatives; Sees Q4 EPS of $(0.02)-(0.03) vs $0.03-(0.03) Prior  -> [Guidance]
13. Kopin Chairman Fan Buys 116,400 Shares @$2.83/Share -Form 4 -> [OTHER]

Given the following headline:

### START HEADLINE ###

{headline}

### END HEADLINE ###

What event type best classifies it? Answer only with your predicted class and give it inside double square brackets, like [[class]]
"""

In [43]:
import asyncio
import aiohttp

async def classify_headline(session, headline):
    """Get classification for a single headline."""
    # Use the PROMPT from the notebook, formatted with the headline
    prompt = PROMPT.format(headline=headline)
    payload = {
        "model": "meta/llama-3.3-70b-instruct",
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "max_tokens": 100,
        "temperature": 0.1
    }
    
    async with session.post("http://nim.test/v1/chat/completions", json=payload) as response:
        result = await response.json()
        return result['choices'][0]['message']['content']

async def process_dataset(dataset, num_samples=None, concurrent=10):
    """Process headlines and return DataFrame with headline and predicted_label."""
    # Access headlines directly from the dataset column
    if num_samples is None:
        headlines = dataset['train']['headline']
    else:
        headlines = dataset['train']['headline'][:num_samples]
    
    semaphore = asyncio.Semaphore(concurrent)
    
    async def classify_with_limit(session, headline):
        async with semaphore:
            label = await classify_headline(session, headline)
            return {"headline": headline, "predicted_label": label}
    
    async with aiohttp.ClientSession() as session:
        tasks = [classify_with_limit(session, h) for h in headlines]
        results = await asyncio.gather(*tasks)
    
    return pd.DataFrame(results)


In [ ]:
# For this setup we will use 50000 samples
df = await process_dataset(data, num_samples=50000, concurrent=100)
df

In [ ]:
# To process the full dataset (or a larger sample):
# df_full = await process_dataset(data, num_samples=None, concurrent=10)
# df_full.to_csv('classified_headlines.csv', index=False)
